# Compile the model

In [1]:
from keras import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras import optimizers

Using TensorFlow backend.
/home/niraj/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
model = Sequential()
model.add(Flatten(input_shape=(7,7,512)))
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [3]:
adam = optimizers.Adam(lr=1e-6)

In [4]:
model.compile(optimizer= adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Import features from bcolz store

In [5]:
import bcolz

In [6]:
train_features = bcolz.open(rootdir='train_store', mode='r')
val_features = bcolz.open(rootdir='val_store', mode='r')

In [7]:
train_features.shape

(23000, 7, 7, 512)

In [8]:
val_features.shape

(2000, 7, 7, 512)

# Import labels from pickle file

In [9]:
import pickle

In [10]:
with open("train_labels.pkl", "rb") as f:
    train_labels = pickle.load(f)
f.close()

with open("val_labels.pkl", "rb") as f:
    val_labels = pickle.load(f)
f.close()

In [11]:
train_labels

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [12]:
val_labels

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

# Initialize keras callbacks

In [13]:
from keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger, EarlyStopping

In [14]:
tb = TensorBoard(log_dir='./logs', batch_size=64, write_images=True, write_graph=True)

In [15]:
mc = ModelCheckpoint('./models/weights.{epoch:02d}-{val_loss:.3f}.hdf5', save_best_only=True)

In [16]:
cl = CSVLogger('./CSV/baseline.csv', separator=',')

In [17]:
es = EarlyStopping(monitor='val_loss', patience=5)

# Fit the model to the features

In [19]:
model.fit(train_features, 
          train_labels, 
          batch_size=64, 
          epochs=10, 
          callbacks = [tb, mc, cl, es],
          validation_data=(val_features, val_labels),
         )

Train on 23000 samples, validate on 2000 samples
Epoch 1/10
23000/23000 [==============================] - 27s 1ms/step - loss: 0.5203 - acc: 0.9117 - val_loss: 0.2004 - val_acc: 0.9675
Epoch 2/10
23000/23000 [==============================] - 26s 1ms/step - loss: 0.0988 - acc: 0.9798 - val_loss: 0.1753 - val_acc: 0.9690
Epoch 3/10
23000/23000 [==============================] - 26s 1ms/step - loss: 0.0353 - acc: 0.9938 - val_loss: 0.1745 - val_acc: 0.9675
Epoch 4/10
23000/23000 [==============================] - 26s 1ms/step - loss: 0.0181 - acc: 0.9984 - val_loss: 0.1691 - val_acc: 0.9695
Epoch 5/10
23000/23000 [==============================] - 26s 1ms/step - loss: 0.0153 - acc: 0.9990 - val_loss: 0.1663 - val_acc: 0.9705
Epoch 6/10
23000/23000 [==============================] - 27s 1ms/step - loss: 0.0142 - acc: 0.9990 - val_loss: 0.1645 - val_acc: 0.9710
Epoch 7/10
22976/23000 [============================>.] - ETA: 0s - loss: 0.0140 - acc: 0.9991

RuntimeError: Problems closing file (file write failed: time = Sun Mar 11 12:07:18 2018
, filename = './models/weights.07-0.163.hdf5', file descriptor = 65, errno = 28, error message = 'No space left on device', buf = 0x5d78ed0, total write size = 2048, bytes this sub-write = 2048, bytes actually written = 18446744073709551615, offset = 4096)

# TODO

* Try data augmentation
* Try adding dropout and batch normalization
* L2 Regualizer maybe?
* Submit